# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  6.00it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.64it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.22it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.07it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.23it/s]



  0%|          | 0/23 [00:00<?, ?it/s]

...

 83%|████████▎ | 19/23 [00:05<00:00,  4.36it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  4.39it/s]

 91%|█████████▏| 21/23 [00:06<00:00,  4.55it/s]

 96%|█████████▌| 22/23 [00:06<00:00,  4.55it/s]

100%|██████████| 23/23 [00:06<00:00,  3.53it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Shelby and I am a senior at the University of North Georgia (UNG). I am a part of the Army ROTC program and will be commissioning as a Second Lieutenant in the United States Army. I am also a member of the UNG Diving Team and a member of the UNG community service organization, Alpha Phi Omega.
As a student in the Army ROTC program, I have had the opportunity to participate in various leadership development training courses. Some of these courses include Cadet Leadership Development, Advanced Leadership Development, and the Leader Development and Assessment Course (LDAC). I have also had the opportunity to lead a platoon of
Prompt: The president of the United States is
Generated text:  not immune from the law. The president can be sued for actions taken while in office. However, there are some exceptions to the president's immunity from prosecution.
The President is entitled to certain protections from prosecution, including executive privilege

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Karen

,

 and

 I

...

 to

 us

,

 but they

 are

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Sarah and I am a UX Designer. I recently worked with a startup to redesign their website, and I am looking for feedback on my work. Specifically, I am seeking feedback on the following aspects:

1.  **Usability**: Is the website intuitive and easy to navigate? Are the main elements and actions clear and accessible?
2.  **Information Architecture**: Does the website's information architecture make sense and effectively organize content?
3.  **Visual Design**: Is the visual design aesthetically pleasing, and does it effectively communicate the brand's identity?
4.  **Accessibility**: Has the website been designed with accessibility in mind,

Prompt: The capital of France is
Generated text:  known for its stunning beauty, rich history, and vibrant culture. Whether you're interested in art, architecture, cuisine, or fashion, Paris has something to offer. From the iconic Eiffel Tower to the stunning Notre-Dame Cathedral, the City of Light is a mus

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Bob

bi

 and

 I

...

ators

,

 entrepreneurs

 and

 techn

In [6]:
llm.shutdown()